<a href="https://colab.research.google.com/github/AhmedZaghl0ul/Machine-Learning/blob/main/Face_Mask_Detection_(Part_1)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow

In [1]:
import os
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/DataSets/Face\ Mask\ Detection\ \(2\ Cat\)/

/content/drive/MyDrive/DataSets/Face Mask Detection (2 Cat)


In [5]:
Dic = '/content/drive/MyDrive/DataSets/Face Mask Detection (2 Cat)'
cat = os.listdir(Dic)
print(cat)

['mask_weared_incorrect', 'with_mask', 'without_mask']


In [7]:
from pickle import load
data = []
labels = []
for c in cat :
  path = os.path.join(Dic,c)
  for file in os.listdir(path):
    img_path = os.path.join(path,file)
    img = load_img(img_path , target_size=(128,128))
    img = img_to_array(img)
    img = preprocess_input(img)
    data.append(img)
    labels.append(c)

In [8]:
from tensorflow.keras.utils import to_categorical
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [9]:
data = np.array(data , dtype='float32')
labels = np.array(labels)

In [10]:
(xtrain , xtest , ytrain , ytest) = train_test_split(data , labels , test_size=0.2 , stratify=labels , random_state=42)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=20 ,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode='nearest'
                         )

In [17]:
baseModel = MobileNetV2(weights='imagenet' , include_top=False , input_tensor=Input(shape=(128,128,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4,4))(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(128 , activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3 , activation='softmax')(headModel)

In [19]:
model = Model(inputs=baseModel.input , outputs=headModel)

In [20]:
for l in baseModel.layers:
  l.trainable=False

In [21]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

opt = Adam(learning_rate=INIT_LR)
model.compile(loss='categorical_crossentropy' , optimizer=opt , metrics=['accuracy'])

In [22]:
h = model.fit(
    aug.flow(xtrain , ytrain , batch_size=BS),
    steps_per_epoch=len(xtrain) // BS,
    validation_data=(xtest , ytest),
    validation_steps=len(xtest) // BS,
    epochs=EPOCHS
)

Epoch 1/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 83s 288ms/step - accuracy: 0.5806 - loss: 1.1609 - val_accuracy: 0.9154 - val_loss: 0.2279
Epoch 2/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8750 - loss: 0.3689 - val_accuracy: 0.9165 - val_loss: 0.2262
Epoch 3/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 30s 134ms/step - accuracy: 0.8896 - loss: 0.2898 - val_accuracy: 0.9377 - val_loss: 0.1632
Epoch 4/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8750 - loss: 0.2453 - val_accuracy: 0.9371 - val_loss: 0.1622
Epoch 5/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 32s 143ms/step - accuracy: 0.9194 - loss: 0.2208 - val_accuracy: 0.9477 - val_loss: 0.1421
Epoch 6/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9375 - loss: 0.1195 - val_accuracy: 0.9477 - val_loss: 0.1420
Epoch 7/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 30s 135ms/step - accuracy: 0.9412 - loss: 0.1749 - val_accuracy: 0.9527 - val_loss: 0.1287
Epoch 8/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9688 - loss: 0.2146 - val_

In [23]:
pred = model.predict(xtest , batch_size=BS)
print(pred)
print("===============================================")
pred = np.argmax(pred , axis=1)
print(pred)

57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 216ms/step
[[1.0891397e-06 3.2123480e-05 9.9996674e-01]
 [3.5242900e-07 1.6821321e-06 9.9999797e-01]
 [3.4099870e-08 2.6018174e-06 9.9999738e-01]
 ...
 [7.8074648e-03 9.9202019e-01 1.7226685e-04]
 [1.4045426e-03 9.9858296e-01 1.2493414e-05]
 [6.6009447e-06 1.7050861e-05 9.9997640e-01]]
[2 2 2 ... 1 1 2]


In [24]:
from sklearn.metrics import classification_report
print(classification_report(ytest.argmax(axis=1) , pred , target_names=lb.classes_))

                       precision    recall  f1-score   support

mask_weared_incorrect       0.97      0.97      0.97       599
            with_mask       0.97      0.97      0.97       599
         without_mask       0.99      0.99      0.99       599

             accuracy                           0.98      1797
            macro avg       0.98      0.98      0.98      1797
         weighted avg       0.98      0.98      0.98      1797



In [27]:
model.save("Mask_Detector.h5")